In [1]:
!pip install vaderSentiment requests gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 113.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.6 MB/s eta 0:00:00


In [2]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import requests
import json
import gradio as gr
import random
import getpass

In [3]:
# Initialize sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

In [4]:
# Request API KEY
API_KEY = getpass.getpass("Enter your Gemini API key: ")
ENDPOINT = "https://generativelanguage.googleapis.com/v1/models/gemini-1.5-flash:generateContent"

Enter your Gemini API key: ··········


In [5]:
def get_sentiment(text):
    """Analyze sentiment using VADER."""
    score = analyzer.polarity_scores(text)
    compound = score['compound']
    if compound >= 0.05:
        return "positive"
    elif compound <= -0.05:
        return "negative"
    else:
        return "neutral"

In [19]:
import requests

def generate_bad_advice(sentiment):
    if sentiment == "positive":
        prompt = "Imagine a bad guy who’s bored who lives in a place that all drugs are legalized. Give them advice on how to have a fun night involving drugs, gambling, drinking, and cheap strip clubs. Encourage them to buy things they don't need, like lottery tickets and luxury goods. Keep it brief, to a few sentences at a maximum"
    elif sentiment == "negative":
        prompt = "Pretend you're a bad friend and tell someone they should just give up because nothing in life ever improves and they're a big fat failure."
    else:
        prompt = "Imagine a confused person. Pretend to be a jerk and tell them they're an idiot who constantly says meaningless things and never does anything of value."

    payload = {
        "contents": [{
            "parts": [{"text": prompt}]
        }]
    }

    headers = {"Content-Type": "application/json"}
    response = requests.post(f"{ENDPOINT}?key={API_KEY}", headers=headers, json=payload)

    if response.status_code == 200:
        result = response.json()
        return result["candidates"][0]["content"]["parts"][0]["text"]
    else:
        return f"API Error: {response.status_code} - {response.text}"

In [20]:
# Main chatbot function
def bad_mental_health_chatbot():
    print("Welcome to the Worst Mental Health Chatbot Ever!")
    while True:
        user_input = input("\nHow are you feeling today? (or type 'exit' to quit): ")
        if user_input.lower() == "exit":
            print("Fine, leave. See if I care.")
            break
        sentiment = get_sentiment(user_input)
        advice = generate_bad_advice(sentiment)
        print(f"Sentiment detected: {sentiment}")
        print(f"Advice: {advice}")

In [21]:
def chatbot_response(user_input):
    sentiment = get_sentiment(user_input)
    advice = generate_bad_advice(sentiment)
    return f"Sentiment: {sentiment}\nAdvice: {advice}"

In [22]:
# Create Gradio interface
interface = gr.Interface(
    fn=chatbot_response,
    inputs=gr.Textbox(label="How are you feeling today?"),
    outputs=gr.Textbox(label="Worst Advice Ever"),
    title="Bad Mental Health Chatbot",
    description="Tell me how you feel, and I’ll give you the worst possible advice!"
)

# Launch the app
interface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://df2d51261f726e1480.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [18]:
bad